### Silbentrennung [Bwinf44 2025](https://bwinf.de/bundeswettbewerb/44/) - J2

[Video](https://youtu.be/kqe2_WPfZwk)



<img src='silbentrennung.png' width='250'>

#### Beispieldaten

Jede Datei enthält einen Text, den Lars nach Silben aufteilen möchte. 
Wenn Lars lediglich seine eigenen Regeln anwenden würde, wäre das Ergebnis für silben05.txt diese Aufteilung:

Se-i-n Va-te-r ist Ba-u-e-r und er-nte-t Ma-i-s.

In [21]:
# Anschauen der Beispieldaten 01-08
nr = '01'
print(f'Beispiel {nr}:')
f = open(f'beispieldaten/silben{nr}.txt', encoding='utf-8')
print(f.read().strip())
f.close()

Beispiel 01:
Mein Name ist Lars und ich esse schrecklich gerne Sauerkraut.


#### Lösungsidee

Wir setzen zunächst die Regeln von Lars um. Für jede Regel ermitteln wir eine Trennliste.
Gegeben ein *wort*, dann gilt für die *trennliste*: 

    trennliste[k] = -1, falls nach wort[k] nicht getrennt werden darf
    trennliste[k] = 1, falls nach wort[k] getrennt werden darf
    trennliste[i] = 0, falls die Regel keine Information zur Trennung nach wort[k] liefert.

Dann gehen wir die trennlisten in der Reihenfolge ihrer Wichtigkeit durch.

<img src='bild0.png' width='400'>

In [22]:
[0, 1, 1, 0, 1, 0, 0, 1, 1, 0]

[0, 1, 1, 0, 1, 0, 0, 1, 1, 0]

In [23]:
for x in trennlisten:
    print(x)

[-1, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0]
[0, -1, 0, 0, 0, 0, 0, -1, 0, 0, 0]
[-1, 0, 0, 0, -1, 1, -1, 0, 1, -1, 0]
[0, 1, 1, -1, 0, 0, 0, 1, 1, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]


#### Umsetzung

Zunächst definieren wir uns ein paar Variablen. Jedes Zeichen, was kein Buchstabe ist, ist ein Satzzeichen.


In [24]:
vokale = 'aeiouäöüAEIOUÄÖÜ'
konsonanten = 'bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ'
buchstaben = vokale + konsonanten

Wir setzen jede Regel in einer eigenen Funktion um und überprüfen diese mit ein paar Beispielen.



##### Regel1

In [25]:
def regel1(wort):
    ''' Ein Wort kann zwischen zwei Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    for i in range(0,len(wort) - 1):
        if wort[i] in konsonanten and wort[i + 1] in konsonanten:
            tmp[i] = 1
    return tmp

In [26]:
regel1('Sauerkraut')

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0]

In [27]:
def trennung(wort,trennliste):
    ''' returns: die Trennung von wort an den 1er-Stellen der trennliste '''
    tmp = ''
    for i in range(len(wort)):
        if trennliste[i] == 1:
            tmp += wort[i]+'-'
        else:
            tmp +=  wort[i]
    return tmp

In [28]:
wort = 'Sauerkraut'
trennung(wort, regel1(wort))

'Sauer-k-raut'

In [29]:
nr = '03'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()
for wort in satz.split():
    print(trennung(wort, regel1(wort)), end=' ')

Es is-t ar-s-c-h-kal-t! 

##### Regel 2

Bei Regel 2 müssen wir berücksichtigen, dass das letzte Zeichen in dem gesplitteten Satz ein Satzzeichen sein kann.

In [30]:
def regel2(wort):
    '''Erster und letzter Buchstabe eines Wortes dürfen nicht abgetrennt werden '''
    tmp = [0]*(len(wort))
    tmp[0] = -1
    if wort[-1] in buchstaben: 
        tmp[-2] = -1
    else:
        tmp[-3] = -1
    return tmp


In [31]:
regel2('Sauerkraut')

[-1, 0, 0, 0, 0, 0, 0, 0, -1, 0]

##### Regel 3

*Bei drei aufeinanderfolgenden Konsonanten kann nur nach dem ersten Konsonanten getrennt werden.*

Das bedeutet *ctsa* kann nur *c-tsa* getrennt werden. Was bedeutet das, wenn mehr als drei Konsonanten aufeinander folgen? Auch da kann nur nach dem ersten Konsonanten getrennt werden: *ctsva* kann nur *c-tsva* getrennt werden.

In [32]:
def regel3(wort):
    '''bei drei aufeinanderfolgenden Konsonanten kann nur nach dem ersten Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    i = 0
    while i < len(wort)-2:
         if wort[i] in konsonanten and wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
             tmp[i] = 1
             i+=1
             while i < len(wort) and wort[i] in konsonanten:
                 tmp[i] = -1
                 i+=1
         i+=1
    return tmp

In [33]:
wort = 'bbb'
trennung(wort, regel3(wort))

'b-bb'

In [34]:
nr = '01'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()
for wort in satz.split():
    print(trennung(wort, regel3(wort)), end=' ')

Mein Name ist Lars und ich esse s-chrec-klich gerne Sauer-kraut. 

##### Regel 4

In [35]:
def regel4(wort):
    ''' Ein Wort kann nach einem Vokal getrennt werden, falls nicht zwei Konsonanten folgen '''
    tmp = [0]*(len(wort))
    for i in range(len(wort) - 2):
        if wort[i] in vokale:
            if wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
                tmp[i] = -1
            else:
                tmp[i] = 1            
    k = len(wort)-2
    if wort[k] in vokale and wort[k + 1] in buchstaben: 
        tmp[k] = 1

    return tmp

In [36]:
nr = '01'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()
for wort in satz.split():
    print(trennung(wort, regel4(wort)), end=' ')

Me-i-n Na-me ist Lars und ich esse schrecklich gerne Sa-u-erkra-u-t. 

#### Alle Regeln zusammenfassen

In [37]:
wort = 'Sauerkraut'
trennlisten = [regel4(wort), regel3(wort), regel2(wort), regel1(wort)]
trennlisten

[[0, 1, 1, -1, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 1, -1, -1, 0, 0, 0],
 [-1, 0, 0, 0, 0, 0, 0, 0, -1, 0],
 [0, 0, 0, 0, 1, 1, 0, 0, 0, 0]]

In [38]:
def trennliste_gesamt(trennlisten):
    n = len(trennlisten[0])
    tmp = [0]*n
    for i in range(n):
        for t in trennlisten:
            if t[i] == -1: break
            if t[i] == 1:
                tmp[i] = 1
                break
    return tmp

In [39]:
trennliste_gesamt(trennlisten)

[0, 1, 1, 0, 1, 0, 0, 1, 1, 0]

In [40]:
for wort in satz.split():
    trennlisten = [regel4(wort), regel3(wort),regel2(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Me-i-n Na-me ist Lars und ich es-se s-chrec-klich ger-ne Sa-u-er-kra-u-t. 

In [41]:
nr = '05'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
for wort in satz.split():
    trennlisten = [regel4(wort), regel3(wort),regel2(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Se-i-n Va-te-r ist Ba-u-e-r und er-nte-t Ma-i-s. 

#### Das ganze Programm

In [47]:
def regel1(wort):
    ''' Ein Wort kann zwischen zwei Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    for i in range(0,len(wort) - 1):
        if wort[i] in konsonanten and wort[i + 1] in konsonanten:
            tmp[i] = 1
    return tmp

def regel2(wort):
    '''Erster und letzter Buchstabe eines Wortes dürfen nicht abgetrennt werden '''
    erster = 0
    while wort[erster] not in buchstaben and erster < len(wort):
        erster += 1
    letzter = len(wort) - 1
    while wort[letzter] not in buchstaben and letzter > 0:
        letzter -= 1

    tmp = [0]*(len(wort))
    tmp[erster] = -1
    tmp[letzter-1] = -1
    return tmp

def regel3(wort):
    '''bei drei aufeinanderfolgenden Konsonanten kann nur nach dem ersten Konsonanten getrennt werden '''
    tmp = [0]*(len(wort))
    i = 0
    while i < len(wort)-2:
         if wort[i] in konsonanten and wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
             tmp[i] = 1
             i+=1
             while i < len(wort) and wort[i] in konsonanten:
                 tmp[i] = -1
                 i+=1
         i+=1
    return tmp

def regel4(wort):
    ''' Ein Wort kann nach einem Vokal getrennt werden, falls nicht zwei Konsonanten folgen '''
    tmp = [0]*(len(wort))
    for i in range(len(wort) - 2):
        if wort[i] in vokale:
            if wort[i + 1] in konsonanten and wort[i + 2] in konsonanten:
                tmp[i] = -1
            else:
                tmp[i] = 1            
    k = len(wort)-2
    if wort[k] in vokale and wort[k + 1] in buchstaben: 
        tmp[k] = 1

    return tmp

def trennliste_gesamt(trennlisten):
    n = len(trennlisten[0])
    tmp = [0]*n
    for i in range(n):
        for t in trennlisten:
            if t[i] == -1: break
            if t[i] == 1:
                tmp[i] = 1
                break
    return tmp

def trennung(wort,trennliste):
    ''' returns: die Trennung von wort an den 1er-Stellen der trennliste '''
    tmp = ''
    for i in range(len(wort)):
        if trennliste[i] == 1:
            tmp += wort[i]+'-'
        else:
            tmp +=  wort[i]
    return tmp

vokale = 'aeiouäöüAEIOUÄÖÜ'
konsonanten = 'bcdfghjklmnpqrstvwxyzBCDFGHJKLMNPQRSTVWXYZ'
buchstaben = vokale + konsonanten

nr = '01'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()

for wort in satz.split():
    trennlisten = [regel4(wort), regel3(wort),regel2(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Me-i-n Na-me ist Lars und ich es-se s-chrec-klich ger-ne Sa-u-er-kra-u-t. 

#### Regeln von Lars erweitern/ändern

Es ist nicht das Ziel der Aufgabe, die Trennung in jedem Fall korrekt durchzuführen. Wir wollen aber das Ergebnis von klein wenig verbessern. 

Wenn man die Silbentrennungsregeln recherchiert, kommt man bald darauf, dass eine Regel von Lars falsch ist, nämlich die Regel 3. Die sollte richtig heißen: *Bei einer Folge von einem oder mehreren Konsonanten wird immer vor dem letzten getrennt* (und nicht wie in Lars Regel 3 angegeben, nach dem ersten).

Lars Regel 2 sollte höchste Priorität haben.

Einige Buchstabenkombinationen wollen wir nicht trennen (auch wenn es dafür Ausnahmen gibt).

In [43]:
def regel5(wort):
    '''Bei einer Folge von einem oder mehreren Konsonanten wird immer vor dem letzten getrennt'''
    tmp = [0]*(len(wort))
    for i in range(len(wort)):
        if wort[i] in konsonanten:
            tmp[i] = -1
    i = 0    
    while i < len(wort):
         if wort[i] in konsonanten:
            j = i
            while j < len(wort) and wort[j] in konsonanten:
                j+=1
            if j-2 >= 0:
                tmp[j-2] = 1
                i = j
         i+=1
    return tmp
    

<img src='bild1.png' width='300'>

In [44]:
wort = 'abaaabbbaaabbbbbbaab'
trennung(wort,regel5(wort))

'a-baaabb-baaabbbbb-baa-b'

In [45]:
def regel6(wort):
    '''Einige Buchstabenkombinationen wollen wir nicht trennen. '''
    tmp = [0]*(len(wort))
    nicht_trennen = ['ei','ai','ui','eu', 'au', 'äu','ch','sch','ph','rh','sh','th','ck']
    for i in range(len(wort)-1):
        if wort[i]+wort[i+1] in nicht_trennen:
            tmp[i] = -1

    for i in range(len(wort)-2):
        if wort[i]+wort[i+1]+wort[i+2] in nicht_trennen:
            tmp[i] = -1
            tmp[i+1] = -1
        
    return tmp

In [48]:
nr = '01'
f = open(f'beispieldaten/silben{nr}.txt',encoding='utf-8')
satz = f.readline()
f.close()

for wort in satz.split():
    trennlisten = [regel2(wort), regel6(wort), regel5(wort), regel4(wort), regel1(wort)]
    print(trennung(wort, trennliste_gesamt(trennlisten)), end=' ')

Mein Na-me ist Lars und ich es-se sch-reck-lich ger-ne Sau-erk-raut. 

#### Vergleich der Trennungen

    Lars Regeln
    Lars Regeln geändert
    Korrekte Trennung

    01
    Me-i-n Na-me ist Lars und ich es-se s-chrec-klich ger-ne Sa-u-er-kra-u-t.
    Mein Na-me ist Lars und ich es-se sch-reck-lich ger-ne Sau-erk-raut.
    Mein Na·me ist Lars und ich es·se schreck·lich ger·ne Sau·er·kraut.

    02
    De-r Ka-pi-tä-n ist e-ben-fal-ls Prä-si-dent de-r ör-tlic-he-n Dam-pfschif-ffah-rtsge-sel-lschaft. 
    Der Ka-pi-tän ist eben-falls Prä-si-dent der ört-lichen Dampfschiff-fahrts-ge-sellschaft. 
    Der Ka·pi·tän ist eben·falls Prä·si·dent der ört·lichen Dampf·schiff·fahrts·ge·sell·schaft.

    03 
    E-s ist ar-schkalt! 
    Es ist arsch-kalt! 
    Es ist arsch·kalt!

    04
    Ist da-s A-u-di-o-sig-na-l i-m Ra-di-o s-chlec-ht? 
    Ist das Au-di-o-sig-nal im Ra-dio sch-lecht?
    Ist das Au·dio·si·gnal im Ra·dio schlecht?

    05
    Se·i·n Va·te·r ist Ba·u·e·r und er·nte·t Ma·i·s.
    Sein Va-ter ist Bau-er und ern-tet Mais.
    Sein Va·ter ist Bau·er und ern·tet Mais.

    06
    Ich an-gle Kar-pfe-n. 
    Ich ang-le Karp-fen.
    Ich an·gle Karp·fen

    07
    Wa-s sind ac-ht Hob-bits? E-i-n Hob-byte!
    Was sind acht Hob-bits? Ein Hobby-te!
    Was sind acht Hob·bits? Ein Hob·byte!

    08
    Fre-u-de, s-chö-ne-r Göt-ter-fun-ke-n, Toc-hte-r a-u-s E-li-si-u-m, Wi-r bet-re-te-n fe-u-er-trun-ke-n,
    Him-mlis-che, de-i-n He-i-lig-thu-m. De-i-ne Za-u-be-r bin-de-n wi-e-de-r, Wa-s di-e Mo-de s-treng get-he-ilt, 
    Al-le Men-sche-n wer-de-n Brü-de-r, Wo de-i-n san-fte-r Flü-ge-l we-ilt. 

    Freu-de, schö-ner Göt-ter-fun-ken, Toch-ter aus Eli-si-um, Wir bet-re-ten feu-ert-run-ken,
    Himm-lische, dein Hei-ligthum. Dei-ne Zau-ber bin-den wi-e-der, Was die Mo-de st-reng getheilt, 
    Al-le Menschen wer-den Brü-der, Wo dein sanf-ter Flü-gel weilt. 

    Freu·de, schö·ner Göt·ter·fun·ken, Toch·ter aus Eli·si·um, Wir be·tre·ten feu·er·trun·ken, 
    Himm·lische, dein Hei·lig·thum. Dei·ne Zau·ber bin·den wie·der, Was die Mo·de streng ge·theilt,
    Al·le Men·schen wer·den Brü·der, Wo dein sanf·ter Flü·gel weilt. 
    